# Practice Session 08: Data streams

In this session we will take a large corpus of documents and compute some statistics using data streams methods.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

Author: <font color="blue">Àlex Montoya Pérez</font>

E-mail: <font color="blue">alex.montoya.01@estudiant.upf.edu</font>

Date: <font color="blue">22/11/2023</font>

# **Google Colaboratory Setup & Imports**

In order to develop this laboratory, I used Google Colaboratory, since I have worked with different files I had to set up the environment as follows:


1.   Importing the drive module from the google.colab package.
2.   Mounting the Google Drive at the specified path (/content/drive).
3.   Changing the current working directory to the directory where I have all needed data /content/drive/MyDrive/MineriaDadesMasives/Labs/.

Verify that we are in the correct directory:


4.   Printing the current working directory path using !pwd.
5.   Listing the contents of the current directory using !ls.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#Here is how to change current working directory
#By default the current working directory is /content
%cd /content/drive/MyDrive/MineriaDadesMasives/Labs/data/movie_dialog_corpus
#Print path and content of the current directory
!pwd
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/MineriaDadesMasives/Labs/data/movie_dialog_corpus
/content/drive/MyDrive/MineriaDadesMasives/Labs/data/movie_dialog_corpus
movie_lines.tsv.gz  README.md


In [ ]:
import io
import nltk
import gzip
import random
import statistics
import secrets
import re
import gzip


# 0. Dataset and how to iterate

In [ ]:
# Leave this code as-is

INPUT_FILE = "movie_lines.tsv.gz"

In [ ]:
# Leave this code as-is

# Producer in Python that reads a filename by words
def read_by_words(filename, max_words=-1, report_every=-1):

    # Open the input file
    with gzip.open(INPUT_FILE, "rt", encoding='utf8') as file:

        # Initialize counter of words to stop at max_words
        counter = 0

        # Regular expression to identify words having 3 letters or more and beginning with a-z
        word_expr = re.compile('^[a-z]{2,}$', re.IGNORECASE)

        # Iterate through lines in the file
        for line in file:

            elements = line.split("\t")

            text = ""
            if len(elements) >= 5:
                text = elements[4].strip()

            if counter > max_words and max_words != -1:
                break

            for word in nltk.word_tokenize(text):

                if word_expr.match(word):
                    counter += 1

                    # Report
                    if (report_every != -1) and (counter % report_every == 0):
                        if max_words == -1:
                            print("- Read %d words so far" % (counter))
                        else:
                            print("- Read %d/%d words so far" % (counter, max_words))

                    # Produce the word in lowercase
                    yield word.lower()

In [ ]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
  # Leave this code as-is

# Iterate through the file
for word in read_by_words(INPUT_FILE, max_words=300000, report_every=100000):
    # Prints 1/10000 of words
    if random.random() < 0.0001:
        print("Current word '%s'" % (word))

Current word 'image'
Current word 'this'
Current word 'you'
Current word 'up'
Current word 'it'
Current word 'did'
Current word 'who'
- Read 100000/300000 words so far
Current word 'around'
Current word 'something'
Current word 'fake'
Current word 'must'
Current word 'paint'
Current word 'do'
Current word 'same'
Current word 'it'
Current word 'you'
Current word 'fast'
- Read 200000/300000 words so far
Current word 'were'
Current word 'hospital'
Current word 'hall'
Current word 'it'
Current word 'hold'
Current word 'that'
Current word 'me'
Current word 'is'
Current word 'this'
Current word 'doctor'
Current word 'out'
Current word 'catchin'
Current word 'elizabeth'
Current word 'body'
- Read 300000/300000 words so far


In [ ]:
# Run this if above gives an error about 'punkt'
# nltk.download('punkt')

# 1. Determine approximately the top-10 words

## Add Reservoir Function

In [ ]:
# adds an item to the reservoir, maintaining its size. If the reservoir is already of size max_size, a random item is selected and evicted before adding the item. It is important to evict an old item before adding the new item. Use the following skeleton:

def add_to_reservoir(reservoir, item, max_reservoir_size):
    # If reservoir is not full, we just add the item
    if len(reservoir) < max_reservoir_size:
        reservoir.append(item)
    # If it's full, we discard randomly one item and put the new item there.
    else:
        random_index = random.randint(0, max_reservoir_size - 1)
        reservoir[random_index] = item

    assert len(reservoir) <= max_reservoir_size


## Reservoir Sampling

In [ ]:
#function to iterate through the file using the reservoir sampling method seen in class. In this function you will decide, for every item, whether to call add_to_reservoir or to ignore the item.
def reservoir_sampling(filename, reservoir_size, max_words=-1, report_every=-1):
    reservoir = []
    words_read = 0

    for word in read_by_words(filename, max_words=max_words, report_every=report_every):
        words_read += 1

        # If it's not full, call the function
        if len(reservoir) < reservoir_size:
            add_to_reservoir(reservoir, word, reservoir_size)
        else:
            # With probability 1 - s/n we ignore the item
            if random.random() < 1 - reservoir_size/words_read:
                continue
            # With probability s/n we add the item, discarding another item
            else:
                add_to_reservoir(reservoir, word, reservoir_size)

    return (words_read, reservoir)


In [ ]:
# Leave this code as-is

reservoir_size = 1500
(items_seen, reservoir) = reservoir_sampling(INPUT_FILE, reservoir_size, max_words=1000000, report_every=100000)

print("Number of items seen    : %d" % items_seen)
print("Number of items sampled : %d" % len(reservoir) )

- Read 100000/1000000 words so far
- Read 200000/1000000 words so far
- Read 300000/1000000 words so far
- Read 400000/1000000 words so far
- Read 500000/1000000 words so far
- Read 600000/1000000 words so far
- Read 700000/1000000 words so far
- Read 800000/1000000 words so far
- Read 900000/1000000 words so far
- Read 1000000/1000000 words so far
Number of items seen    : 1000028
Number of items sampled : 1500


In [ ]:
# Leave this code as-is

freq = {}
for item in reservoir:
    freq[item] = reservoir.count(item)

most_frequent_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:10]
for absolute_frequency, word in most_frequent_items:
    print("%d %s" % (absolute_frequency, word))

89 you
43 the
35 to
32 it
28 that
28 do
24 and
21 of
19 we
18 what


## Top Items and their Relative Frequencies

In [ ]:
freq = {}
for item in reservoir:
    freq[item] = reservoir.count(item)

# Sort items by absolute frequency in descending order and select the top 15
most_frequent_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:15]

# Print the 15 most frequent items and their relative frequencies as percentages
for absolute_frequency, word in most_frequent_items:
    # Calculate relative frequency as a percentage
    relative_frequency = (absolute_frequency / len(reservoir)) * 100
    # Print the result
    print("%.2f%%" % relative_frequency, word)


5.93% you
2.87% the
2.33% to
2.13% it
1.87% that
1.87% do
1.60% and
1.40% of
1.27% we
1.20% what
1.20% is
1.13% in
1.13% he
0.93% my
0.93% for


## Estimate for the relative and absolute frequency of the words in the entire dataset.

In [ ]:
# Increase the max limit of words so that one pass takes no more than 5 minutes to be completed. Replace this cell with your code to try different reservoir sizes. In each case, print your estimate for the relative and absolute frequency of the words in the entire dataset.
def calculate_frequencies(reservoir):
    freq = {}
    for item in reservoir:
        freq[item] = freq.get(item, 0) + 1
    return freq

def print_top_words(reservoir_size, max_words=1000000, report_every=100000):
    print("Reservoir size:", reservoir_size, "\n")

    items_seen, reservoir = reservoir_sampling(INPUT_FILE, reservoir_size, max_words=max_words, report_every=report_every)
    print("Number of items seen    : %d" % items_seen)
    print("Number of items sampled : %d" % len(reservoir))

    freq = calculate_frequencies(reservoir)

    most_frequent_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:5]
    for absolute_frequency, word in most_frequent_items:
        relative_frequency = absolute_frequency / len(reservoir)
        print("Absolute freq: %d, Relative freq: %f, word: %s" % (absolute_frequency, relative_frequency, word))


### Test different reservoir sizes [100, 500, 3000]

In [ ]:
# Reservoir Size = 100
print_top_words(100)
print("\n")

# Reservoir Size = 500
print_top_words(500)
print("\n")

# Reservoir Size = 3000
print_top_words(3000)
print("\n")

Reservoir size: 100 

- Read 100000/1000000 words so far
- Read 200000/1000000 words so far
- Read 300000/1000000 words so far
- Read 400000/1000000 words so far
- Read 500000/1000000 words so far
- Read 600000/1000000 words so far
- Read 700000/1000000 words so far
- Read 800000/1000000 words so far
- Read 900000/1000000 words so far
- Read 1000000/1000000 words so far
Number of items seen    : 1000028
Number of items sampled : 100
Absolute freq: 7, Relative freq: 0.070000, word: you
Absolute freq: 3, Relative freq: 0.030000, word: just
Absolute freq: 3, Relative freq: 0.030000, word: did
Absolute freq: 3, Relative freq: 0.030000, word: and
Absolute freq: 2, Relative freq: 0.020000, word: what


Reservoir size: 500 

- Read 100000/1000000 words so far
- Read 200000/1000000 words so far
- Read 300000/1000000 words so far
- Read 400000/1000000 words so far
- Read 500000/1000000 words so far
- Read 600000/1000000 words so far
- Read 700000/1000000 words so far
- Read 800000/1000000 words

## Min Reservoir size to have stable results

In [ ]:
#Find by trial and error, and include in your report, the minimum reservoir size you need to have somewhat stable results (e.g., the same top-3 words in two consecutive runs of the algorithm).

# Reservoir Size = 1000
print_top_words(1000)
print("\n")

# Reservoir Size = 1500
print_top_words(1500)
print("\n")

# Reservoir Size = 5000
print_top_words(5000)
print("\n")

Reservoir size: 1000 

- Read 100000/1000000 words so far
- Read 200000/1000000 words so far
- Read 300000/1000000 words so far
- Read 400000/1000000 words so far
- Read 500000/1000000 words so far
- Read 600000/1000000 words so far
- Read 700000/1000000 words so far
- Read 800000/1000000 words so far
- Read 900000/1000000 words so far
- Read 1000000/1000000 words so far
Number of items seen    : 1000028
Number of items sampled : 1000
Absolute freq: 55, Relative freq: 0.055000, word: you
Absolute freq: 35, Relative freq: 0.035000, word: to
Absolute freq: 30, Relative freq: 0.030000, word: that
Absolute freq: 21, Relative freq: 0.021000, word: the
Absolute freq: 21, Relative freq: 0.021000, word: it


Reservoir size: 1500 

- Read 100000/1000000 words so far
- Read 200000/1000000 words so far
- Read 300000/1000000 words so far
- Read 400000/1000000 words so far
- Read 500000/1000000 words so far
- Read 600000/1000000 words so far
- Read 700000/1000000 words so far
- Read 800000/1000000 

Examining the output above reveals distinct top three words when reservoir_size is set to 1000 compared to 1500, but interestingly, the results align with reservoir_size = 5000. This suggests that 5000 might be the optimal minimum reservoir size. While some runs with reservoir_size = 1000 did yield the same three words, the consistency was not as pronounced as with reservoir_size = 1500. Therefore, based on these observations, I would recommend utilizing reservoir_size = 1500.

# 2. Determine approximately the distinct number of words

In [ ]:
# Leave this code as-is

def count_trailing_zeroes(number):
    count = 0
    while number & 1 == 0:
        count += 1
        number = number >> 1
    return count

In [ ]:
# Leave this code as-is

def random_hash_function():
    # We use a cryptographically safe generator for the salt of our hash function
    salt = secrets.token_bytes(32)
    return lambda string: hash(string + str(salt))

## Perform Requested number of passes

In [ ]:
# Function to perform the probabilistic counting and return the estimate
def perform_probabilistic_counting(num_passes, max_words=None):
    estimates = []

    for i in range(num_passes):
        hash_function = random_hash_function()
        R = []
        for word in read_by_words(INPUT_FILE, max_words=max_words, report_every=-1):
            hash_value = hash_function(word)
            R.append(count_trailing_zeroes(hash_value))
        estimate = 2 ** max(R)
        estimates.append(estimate)
        print("Estimate on pass %d: %d distinct words" % (i + 1, estimate))

    return estimates

number_of_passes = 5
estimates_5_passes = perform_probabilistic_counting(number_of_passes, max_words=1000000)

Estimate on pass 1: 16384 distinct words
Estimate on pass 2: 16384 distinct words
Estimate on pass 3: 65536 distinct words
Estimate on pass 4: 131072 distinct words
Estimate on pass 5: 262144 distinct words


In [ ]:
# Leave this code as-is

print("* Average of estimates: %.1f" % statistics.mean(estimates_5_passes))
print("* Median  of estimates: %.1f" % statistics.median(estimates_5_passes))

* Average of estimates: 98304.0
* Median  of estimates: 65536.0


## Comparing Algorithm Performance with 10 Passes vs. 20 Passes in Triple Runs

In [ ]:
# Perform 3 separate runs with 10 passes each
for run in range(3):
    print(f"\nRun {run + 1} (10 passes):")
    estimates_10_passes = perform_probabilistic_counting(num_passes=10, max_words=1000000)
    print(f"Median of estimates: {statistics.median(estimates_10_passes)}")
    print(f"Average of estimates: {statistics.mean(estimates_10_passes)}")

# Perform 3 separate runs with 20 passes each
for run in range(3):
    print(f"\nRun {run + 1} (20 passes):")
    estimates_20_passes = perform_probabilistic_counting(num_passes=20, max_words=1000000)
    print(f"Median of estimates: {statistics.median(estimates_20_passes)}")
    print(f"Average of estimates: {statistics.mean(estimates_20_passes)}")



Run 1 (10 passes):
Estimate on pass 1: 16384 distinct words
Estimate on pass 2: 32768 distinct words
Estimate on pass 3: 4194304 distinct words
Estimate on pass 4: 4096 distinct words
Estimate on pass 5: 16384 distinct words
Estimate on pass 6: 16384 distinct words
Estimate on pass 7: 16384 distinct words
Estimate on pass 8: 8192 distinct words
Estimate on pass 9: 65536 distinct words
Estimate on pass 10: 16384 distinct words
Median of estimates: 16384.0
Average of estimates: 438681.6

Run 2 (10 passes):
Estimate on pass 1: 16384 distinct words
Estimate on pass 2: 32768 distinct words
Estimate on pass 3: 32768 distinct words
Estimate on pass 4: 32768 distinct words
Estimate on pass 5: 32768 distinct words
Estimate on pass 6: 16384 distinct words
Estimate on pass 7: 16384 distinct words
Estimate on pass 8: 8192 distinct words
Estimate on pass 9: 8192 distinct words
Estimate on pass 10: 32768 distinct words
Median of estimates: 24576.0
Average of estimates: 22937.6

Run 3 (10 passes):
E

<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>